In [1]:
# Set dependencies and Setup
import requests
import gmaps
import pandas as pd

# Import API key
from config import g_key

In [2]:
# Configure gmaps ti use your Google API key
gmaps.configure(api_key=g_key)

In [3]:
# set the parameters to search for a hotel in Paris
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"
}

# Use base URL to search for hotels in Paris
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Make a request and get the JSON data from the search
hotels = requests.get(base_url,params=params).json()


In [4]:
len(hotels["results"])

20

In [5]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Husavik,IS,2021-01-27 21:07:40,66.0449,-17.3389,19.40,86,75,8.25
1,1,Lompoc,US,2021-01-27 20:54:10,34.6391,-120.4579,51.80,93,90,13.80
2,2,Weiser,US,2021-01-27 21:08:32,44.2510,-116.9693,46.40,53,75,24.16
3,3,Ushuaia,AR,2021-01-27 21:08:33,-54.8000,-68.3000,41.00,87,75,24.16
4,4,Hofn,IS,2021-01-27 21:08:05,64.2539,-15.2082,25.77,89,1,14.12


In [6]:
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [7]:
# Filter the data set to find the cities that fit the criteria 
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                        (city_data_df["Max Temp"] >= min_temp)]

In [8]:
hotel_df = preferred_cities_df[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Luganville,VU,76.59,-15.5333,167.1667,
10,Rikitea,PF,79.74,-23.1203,-134.9692,
11,Butaritari,KI,80.56,3.0707,172.7902,
13,Arraial Do Cabo,BR,84.20,-22.9661,-42.0278,
14,Sosua,DO,82.40,19.7667,-70.5167,


In [9]:
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for params dictionary 
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our lat and long
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # Make a request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Luganville,VU,76.59,-15.5333,167.1667,The Espiritu Hotel
10,Rikitea,PF,79.74,-23.1203,-134.9692,Pension Maro'i
11,Butaritari,KI,80.56,3.0707,172.7902,Isles Sunset Lodge
13,Arraial Do Cabo,BR,84.20,-22.9661,-42.0278,Pousada Porto Praia
14,Sosua,DO,82.40,19.7667,-70.5167,Piergiorgio Palace Hotel


In [17]:
# Add a heatmap of temperature for the vacation spots
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp}</dd>
</dl>
"""

locations = hotel_df[["Lat","Lng"]]
max_temp = hotel_df["Max Temp"]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(locations,weights=max_temp,dissipating=False,max_intensity=300,point_radius=4)

# Add a marker layer
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))